In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN

In [2]:
# Загрузка данных из CSV файлов
df1 = pd.read_csv('/Users/evgenii/Documents/GitHub/perfconf/perfcong-jmeter-examples/clusterization/data/baseline_more_dispertion/raw_data.csv')
df2 = pd.read_csv('/Users/evgenii/Documents/GitHub/perfconf/perfcong-jmeter-examples/clusterization/data/new_release/raw_data.csv')

# Преобразование startTime в формат даты
df1['startTime'] = pd.to_datetime(df1['startTime'], unit='ms')
df2['startTime'] = pd.to_datetime(df2['startTime'], unit='ms')

In [4]:
# # Функция для загрузки и подготовки данных
# def load_and_prepare_data(file_path):
#     df = pd.read_csv(file_path)
#     df['startTime'] = pd.to_datetime(df['startTime'])
#     df['timestamp'] = (df['startTime'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
#     return df

# # Загрузка данных из CSV файлов
# df1 = load_and_prepare_data('timeseries1.csv')
# df2 = load_and_prepare_data('timeseries2.csv')

# Функция для кластеризации и визуализации
def cluster_and_plot(df, algorithm, title):
    X = df[['startTime', 'responseTime']]
    clustering = algorithm.fit(X)
    
    centers = []
    # K-means
    if hasattr(clustering, 'cluster_centers_'):
        centers = clustering.cluster_centers_
        labels = clustering.labels_
        num_clusters = len(centers)
    # DBSCAN
    else:
        labels = clustering.labels_
        unique_labels = set(labels)
        num_clusters = len(unique_labels) - (1 if -1 in labels else 0)
        core_samples_mask = np.zeros_like(labels, dtype=bool)
        core_samples_mask[clustering.core_sample_indices_] = True
        core_samples = X[core_samples_mask]
        if len(core_samples) > 0:
            centers = np.array([core_samples[labels == label].mean(axis=0) for label in unique_labels if label != -1])
    
    # Визуализация
    plt.figure(figsize=(14, 7))
    sns.scatterplot(x='startTime', y='responseTime', hue=labels, palette='viridis', data=df, legend='full')
    if len(centers) > 0:
        plt.scatter(centers[:, 0], centers[:, 1], c='red', s=200, alpha=0.75, marker='X', label='Centers')
    plt.title(f'{title} - {num_clusters} clusters')
    plt.legend()
    plt.show()
    
    return num_clusters, centers

# Функция для сравнения кластеров
def compare_clusters(num_clusters1, centers1, num_clusters2, centers2, threshold=0.2):
    if num_clusters1 != num_clusters2:
        print(f"Количество кластеров отличается: {num_clusters1} vs {num_clusters2}")
    else:
        print(f"Количество кластеров одинаковое: {num_clusters1}")

    if len(centers1) > 0 and len(centers2) > 0:
        for i in range(min(num_clusters1, num_clusters2)):
            dist = np.linalg.norm(centers1[i] - centers2[i])
            if dist > threshold:
                print(f"Центры кластеров {i} отличаются больше чем на {threshold*100}%: {centers1[i]} vs {centers2[i]} (расстояние = {dist})")
            else:
                print(f"Центры кластеров {i} похожи: {centers1[i]} vs {centers2[i]} (расстояние = {dist})")

# Кластеризация и визуализация для первого временного ряда
kmeans = KMeans(n_clusters=3, random_state=42)
dbscan = DBSCAN(eps=0.2, min_samples=5)

print("Кластеризация первого временного ряда:")
num_clusters1_kmeans, centers1_kmeans = cluster_and_plot(df1, kmeans, "K-Means Clustering for Series 1")
num_clusters1_dbscan, centers1_dbscan = cluster_and_plot(df1, dbscan, "DBSCAN Clustering for Series 1")

# Кластеризация и визуализация для второго временного ряда
print("Кластеризация второго временного ряда:")
num_clusters2_kmeans, centers2_kmeans = cluster_and_plot(df2, kmeans, "K-Means Clustering for Series 2")
num_clusters2_dbscan, centers2_dbscan = cluster_and_plot(df2, dbscan, "DBSCAN Clustering for Series 2")

# Наложение временных рядов друг на друга
plt.figure(figsize=(14, 7))
plt.plot(df1['startTime'], df1['responseTime'], label='Series 1', color='blue')
plt.plot(df2['startTime'], df2['responseTime'], label='Series 2', color='orange')
plt.title('Overlay of Two Time Series')
plt.legend()
plt.show()

# Сравнение кластеров
print("\nСравнение кластеров K-Means:")
compare_clusters(num_clusters1_kmeans, centers1_kmeans, num_clusters2_kmeans, centers2_kmeans)

print("\nСравнение кластеров DBSCAN:")
compare_clusters(num_clusters1_dbscan, centers1_dbscan, num_clusters2_dbscan, centers2_dbscan)


Кластеризация первого временного ряда:


DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Int64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes.Int64DType'>)